### Answer from ChatGPT
A unit-disk embedding of a graph is a way of drawing the graph in the plane such that each vertex is represented by a disk of unit radius and two disks intersect if and only if their corresponding vertices are adjacent in the graph. In other words, it is a way of embedding the graph in the plane such that the vertices are represented by non-overlapping disks of equal radius and the edges are represented by tangent segments between the disks. The concept of unit-disk embedding is important in the study of geometric graph theory and has applications in wireless network design, sensor networks, and other areas.

In [1]:
using Graphs, Plots, GraphRecipes, Optim, Optimisers, Random, ForwardDiff, Combinatorics

global E = [(1, 2), (1, 3),
	(2, 3), (2, 4), (2, 5), (2, 6),
	(3, 5), (3, 6), (3, 7),
	(4, 5), (4, 8),
	(5, 6), (5, 8), (5, 9),
	(6, 7), (6, 8), (6, 9),
	(7,9), (8, 9), (8, 10), (9, 10)]

21-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (3, 5)
 (3, 6)
 (3, 7)
 (4, 5)
 ⋮
 (5, 8)
 (5, 9)
 (6, 7)
 (6, 8)
 (6, 9)
 (7, 9)
 (8, 9)
 (8, 10)
 (9, 10)

In [2]:
edges = Edge.(E)

# create a graph from the vertex and edge lists above
g = SimpleGraphFromIterator(edges)

{10, 21} undirected simple Int64 graph

In [3]:
function dist(a::AbstractVector{T},b::AbstractVector{T}) where T
    return sqrt((a[1]-b[1])^2 + (a[2]-b[2])^2)
end

dist (generic function with 1 method)

In [4]:
function loss_f(x::AbstractMatrix)
    loss = 0.0
    for (a,b) in combinations(1:10,2)
		if (a,b) in E
			prop_dist = dist(x[:,a],x[:,b])
		else
			prop_dist = 1/dist(x[:,a],x[:,b])
		end

		if prop_dist > 0.999
			loss  +=  prop_dist
		end

    end
    return loss
end

loss_f (generic function with 1 method)

In [5]:
function isunitdisk(x::AbstractMatrix{T}) where T
	for (a,b) in combinations(1:10,2)
		if(a,b) in E
			@assert dist(x[:,a],x[:,b]) <= 1.0 "Error, got $x[:,a] and $x[:,b] with distance $(dist(x[:,a],x[:,b]))"
		else
			@assert dist(x[:,a], x[:,b]) > 1.0 "Error, non connected points $a and $b are too close, got distance $(dist(x[:,a],x[:,b]))"
		end
	end
	println("Test passed")
end

isunitdisk (generic function with 1 method)

In [6]:
task_success = false
for _ in 1:100
	#initialize so that points start without getting too close to any
	x0 =  float.(vcat(shuffle(1:100)[1:10]',shuffle(1:100)[1:10]'))
	his = optimize(loss_f,x0,NelderMead())
	if  minimum(his) == 0.0
		@info minimum(his)
		@info Optim.minimizer(his)
		ans = Optim.minimizer(his)
		isunitdisk(ans)
		task_success = true
		minx = min(ans[1,:]...)
		miny = min(ans[2,:]...)
		for coord in eachcol(ans)
			println(coord' .- [minx miny])
		end
		break
	end
end
if !task_success
	println("Did not get correct answer in 100 random initial points, retry")
end

Test passed


┌ Info: 0.0
└ @ Main /home/divingwhale/documents/Julia/ModernScientificComputing/courseworks/week7/shiqinruixu/week7.ipynb:7
┌ Info: [48.95664565706372 48.57979829466359 48.108230512019496 48.76713531110493 48.42172412113497 47.75871658288185 47.36435638828799 48.00441950395791 47.642391753552445 47.4424499286042; 73.48322865053181 74.24220092350585 73.97201869189136 75.1350255158153 74.83281644924801 74.58652878691547 74.4023978286742 75.3450545616642 75.0173685938166 75.89385376500006]
└ @ Main /home/divingwhale/documents/Julia/ModernScientificComputing/courseworks/week7/shiqinruixu/week7.ipynb:8


[1.5922892687757297 0.0]
[1.2154419063756023 0.7589722729740345]
[0.7438741237315085 0.4887900413595503]
[1.4027789228169425 1.6517968652834867]
[1.0573677328469842 1.3495877987162004]
[0.3943601945938653 1.1033001363836519]
[0.0 0.9191691781423827]
[0.6400631156699248 1.8618259111323852]
[0.2780353652644578 1.534139943284785]
[0.07809354031621041 2.410625114468246]


### Credits to Yusheng Zhao